In [ ]:
import numpy as np
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf

from scipy import stats
from patsy import dmatrices

import matplotlib.pyplot as plt
import seaborn as sns

# Problem współliniowości

### Diagnostyka

1. **Macierz korelacji predyktorów** - $D_X = (\rho_{ij})$;

2. **Uwarunkowanie macierzy** $\frac{\lambda_{\text{max}}(D_X)}{\lambda_{\text{min}}(D_X)}$ - duże $\implies$ istnieje para predyktorów zależnych liniowo;

3. **VIF** (ang. *variance inflation factor*) - współczynnik podbicia wariancji

    Dla $1\leq i \leq p-1$: $$R^2_i = \frac{\text{RSS}}{\text{TSS}}$$ dla modelu $x_i \sim x_{-i}$, gdzie $x_{-i}$ oznacza wszystkie zmienne objaśniające z  pominięciem $i$-tej.

    Wówczas
    $$
    \text{VIF}_i = \frac{1}{1-R_i^2}
    $$

    **Interpretacja:** Duża wartość dla pewnego $i$ wskazuje na potencjalną liniową zależność $i$-tej zmiennej objaśniającej od pozostałych zmiennych. 

    **Reguła kciuka:** Jeśli $\text{VIF}_i\geq 10$, to $i$-tą zmienną uznajemy w przybliżeniu liniowo zależną od pozostałych.

# Zadanie 1
Dla danych `Carseats` sprawdź, czy występuje w nich problem współliniowości przy użyciu powyższych metod. Jeśli tak, odrzuć ze zbioru zmienne zależne liniowo i dopasuj model regresji liniowej bez nich. Porównaj wyniki.

In [ ]:
# carseats = sm.datasets.get_rdataset(dataname="Carseats", package="ISLR", cache=True)

In [ ]:
# carseats.data

In [ ]:
# Xvar = carseats.data.loc[:,~carseats.data.columns.isin(['Sales'])]
# features = Xvar.columns
# correlation_matrix = Xvar.corr()
# print(correlation_matrix)

In [ ]:
# ##zrobimy heatmapę od wartości bezwzględnych 0 - brak koralcji,  1-mocna korelacja
# import matplotlib as mpl
# heatmap = plt.pcolor(np.abs(correlation_matrix), cmap=mpl.cm.coolwarm, alpha=0.8)

# heatmap.axes.set_frame_on(False)
# heatmap.axes.set_yticks(np.arange(correlation_matrix.shape[0]) + 0.5, minor=False)
# heatmap.axes.set_xticks(np.arange(correlation_matrix.shape[1]) + 0.5, minor=False)
# heatmap.axes.set_xticklabels(features, minor=False)
# plt.xticks(rotation=90)
# heatmap.axes.set_yticklabels(features, minor=False)
# plt.tick_params(axis='both', which='both', bottom='off', 
#                     top='off', left='off', right='off')
# plt.colorbar()
# plt.show()

In [ ]:
# #Vify
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# columns = list(carseats.data.columns)
# columns.remove('Sales')
# features = "+".join(columns)
# y, X = dmatrices('Sales ~ ' + features+"-1", data=carseats.data, return_type='dataframe')

# vif = pd.DataFrame()
# vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
# vif["features"] = X.columns
# vif

# Zadanie 2
Wczytaj dane `kc_house_data.csv` ([This dataset contains house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015](https://www.kaggle.com/harlfoxem/housesalesprediction/data)).

Dopasuj model `price ~ bathrooms + sqft_living + sqft_lot + sqft_above + sqft_basement + lat + long`, uwzględnij współliniowość predyktorów.

In [ ]:
# house = pd.read_csv('kc_house_data.csv')
# house.head()

In [ ]:
# columns = np.array(['bathrooms', 'sqft_living', 'sqft_lot',
#                     'sqft_above', 'sqft_basement', 'lat', 'long'])
# lm = smf.ols('price~bathrooms+sqft_living+sqft_lot+sqft_above+sqft_basement+lat+long',data = house).fit()

In [ ]:
# lm.summary()

In [ ]:
# sns.heatmap(abs(house.corr()),cmap=mpl.cm.coolwarm, alpha=0.8)

In [ ]:
# # #Vify
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# vif = pd.DataFrame()
# vif["VIF Factor"] =[variance_inflation_factor(house[columns].values,i) for i in range(len(columns))]
# vif["features"] = columns
# vif

# Zadanie 3
Wczytaj zbiór `Hald.csv`. Znajdź najlepszy model regresji liniowej uwzględniając współliniowość predyktorów.

Opis zbioru:

    Heat evolved during setting of 13 cement mixtures of four basic ingredients. Each ingredient percentage appears to be rounded down to a full integer. The sum of the four mixture percentages varies from a maximum of 99% to a minimum of 95%. If all four regressor X-variables always summed to 100%, the centered X-matrix would then be of rank only 3. Thus, the regression of heat on four X-percentages is ill-conditioned, with an approximate rank deficiency of MCAL = 1. The first column is the response and the remaining four columns are the predictors.

In [ ]:
# hald = pd.read_csv("Hald.csv")
# hald.head()

# Obserwacje odstające

### Diagnostyka

Obserwacja odstająca (ang.outlier) jest obserwacją, która nie spełnia równania regresji czyli nie należy do modelu regresji. Obserwacje odstające mogą znacząco wpływać na postać prostej regresji.

**Rezyduum** $e_i$ przyjmuje dla $i$-tej obserwacji wartość różnicy:
$$
e_i = y_i - \hat{y}_i.
$$

**Błąd standardowy** takiego rezyduum $e_i$ jest równy:
$$
\text{SE}(e_i) = S\cdot\sqrt{1-h_i},
$$
gdzie 
- $S = \sigma$ oznacza przęciętne odchylenie wartości rzeczywistych od wartości przewidywanych,
- $h_i$ - wartość wpływu $i$-tej obserwacji, która wyraża się wzorem
$$
h_i = \frac{1}{n} + \frac{(x_i - \overline{x})^2}{\sum_{i=1}^n(x_i - \overline{x})^2}
$$

Obserwacje odstajacę dzielimy na 
    - wpływowe - obserwacja jest wpływowa jesli jej usuniecie z modelu ma duży wpływ na dopasowanie modelu/prognoże na podstawie modelu;
    - niewpływowe - obserwacja jest niewpływowa jesli jej usuniecie z modelu nie ma wpływu na dopasowanie modelu/prognoże na podstawie modelu;
  

  
### Detekcja obserwacji odstających:

1. **Wykres studentyzowanych rezyduów**

Dla małych prób, wartości zmiennej objaśniającej nie są w miarę równomiernie rozłożone i niektóre błędy $\text{SE}(e_i)$ mogą znacznie odbierać od błędu $S$. Wówczas dobrze jest analizować rezydua przy użyciu tzw. **rezyduów studentyzowanych**.

$$r_i =\frac{e_i}{\text{SE}(e_i)}$$

To pozwoli wykrywać obserwacje faktycznie odstające, pomijając te, które przy analizie rezyduów $e_i$ sugerowały, że są odstające mimo, że takimi nie były. Dla rezyduów studentyzowanych zakłada się, że przy poziomie ufności równym 0.95 uznaje się je za normalne (zachowujące własność rozkładu normalnego), gdy należą do przedziału $[−2,+2]$.

Wykres studentyzowanych rezyduów względem ich indeksu identyfikuje duże wartości, które przypuszczalnie odpowiadają obserwacjom odstającym. Metodata nie sprawdzi się w sytuacji, gdy mamy w analizowanym zbiorze obserwację wpływową o małej wartości $e_i$. Wówczas bowiem nie określimy jej jako odstającej mimo, że taka w istocie jest.

2. **Wpływowość**

Wpływ $i$-tej obserwacji $h_i$ określamy wzorem
$$
h_i = \frac{1}{n} + \frac{(x_i - \overline{x})^2}{\sum_{i=1}^n(x_i - \overline{x})^2},
$$ 
który określa odstępstwo $x_i$ od $\overline{x}$.

Dla modelu o $p$ parametrach (gdzie $p$ to łączna liczba zmiennych objaśniających i objaśnianych), obserwację uznajemy za wpływową jeśli 
$$
h_i \geq \frac{2p}{n}.
$$

3. **Odległość Cooka**

Jest to miara stopnia zmiany współczynników regresji, gdyby dany przypadek pominąć w obliczeniach współczynników:
$$
D_i = \frac{\sum_{j=1}^n(\hat{Y}_j - \hat{Y}_{j(i)})^2}{pS^2},
$$
gdzie $\hat{Y}_j$ - prognoza na podstawie pełnych danych, $\hat{Y}_{j(i)}$ - prognoza bez $i$-tej obserwacji.

**Interpretacja**: Duża wartość $D_i$ wskazuje na znaczy wpływ usunięcia $i$-tej obserwacji, czyli $i$-ta obserwacja jest obserwacją wpływową.

Wszystkie wartości dla danej odległości powinny być tego samego rzędu. Jeśli tak nie jest, to prawdopodobnie dany przypadek ma istotnie duży wpływ na obciążenie równania regresji.

**Reguła kciuka**: $D_i > \frac{4}{(n − p − 1)}$

# Zadanie 4
Przeanalizuj obserwacje odstające w modelu `model` dla danych `Carseats`. Zidentyfikuj obserwacje im odpowiadające, usuń je ze zbioru i zbuduj model ponownie. Porównaj dopasowanie modeli. 
Analogicznie postępuj dla modelu `model2`.

In [ ]:
# carseats = sm.datasets.get_rdataset(dataname="Carseats", package="ISLR", cache=True)

In [ ]:
# carseats_df = carseats.data

In [ ]:
# columns = list(carseats_df.columns)
# columns.remove('Sales')
# features = "+".join(columns)
# print(features)
# model = smf.ols('Sales~'+features,data = carseats_df)
# fitted = model.fit()
# fitted.summary()

In [ ]:
# h = fitted.get_influence()
# np.where(np.abs(h.resid_studentized)>2)

In [ ]:
# ##wartości odstające
# x = np.arange(1, h.resid_studentized.size + 1)
# plt.scatter(x=x, y=h.resid_studentized)
# plt.hlines(xmin=1, xmax=h.resid_studentized.size + 1, y=-2, color="r")
# plt.hlines(xmin=1, xmax=h.resid_studentized.size + 1, y=0, color="r")
# plt.hlines(xmin=1, xmax=h.resid_studentized.size + 1, y=2, color="r")
# ##te poza pasem kwalifikujemy jako do usunięcia

In [ ]:
# ##wartości wpływowe h_i
# h.hat_matrix_diag
# threshold = 2*h.exog.shape[1]/h.exog.shape[0]

# x = np.arange(1, h.hat_matrix_diag.size + 1)
# plt.scatter(x=x, y=h.hat_matrix_diag)
# plt.hlines(xmin=1, xmax=h.hat_matrix_diag.size + 1, y=threshold, color="r")
# ##są dwie obs wpływowe

In [ ]:
# ##obserwacje wpływowe odległości cooka (inna metoda detekcji wpływowości)
# threshold = 4/(h.exog.shape[0]-h.exog.shape[1]-1)
# h.cooks_distance
# x = np.arange(1, h.cooks_distance[0].size + 1)
# plt.scatter(x=x, y=h.cooks_distance[0])
# plt.hlines(xmin=1, xmax = x[-1], y = threshold, color='r')